## Implementation of Linear Adapters

Linear Adapters involve the training of an additional neural network model that would linearly transform the query dense embeddings such that it would be closer to the chunks that would produce the best context in answering the question. The model will be trained on chunks that exist within the dataset, from which an LLM will be tasked to generate sample questions that would associate with each respective chunk. Therefore, the implementation of this module requires 3 main functions: 
1. Obtaining a train/validation/test dataset
2. Calling the LLM to generate sample questions
3. Training the Linear Adapter

Implementation references the following page: https://www.llamaindex.ai/blog/fine-tuning-a-linear-adapter-for-any-embedding-model-8dd0a142d383

To note: The below code was done in Sprint 2 and was experimental, purely following LlamaIndex's implementation. Edits to implementation were done directly in the class implementation, whereby we no longer use the dataset generated by LlamaIndex, instead using our own dataset generation code. We also edit the training code to use the embedding values within milvus, instead of re-embedding each chunk during training.

In [1]:
# Make Haystack silent
import logging

# Set logging level for specific loggers
logging.getLogger("haystack").setLevel(logging.FATAL)
logging.getLogger("llama_index").setLevel(logging.FATAL)
logging.getLogger("httpx").setLevel(logging.FATAL)
logging.getLogger("openai._base_client").setLevel(logging.FATAL)

In [15]:
import os
from dotenv import load_dotenv
from haystack.components.generators import AzureOpenAIGenerator
from milvus_haystack import MilvusDocumentStore
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI

load_dotenv()

document_store = MilvusDocumentStore(
    collection_name="test_indexing_pipeline",
    collection_description="",
    connection_args={
        "host": os.getenv("MILVUS_HOST", "localhost"),
        "port": os.getenv("MILVUS_PORT", "19530"),
        "user": "",
        "password": "",
        "secure": False,
    },
)

llm = AzureOpenAIGenerator()

llm_llama_index = AzureOpenAI(
    model=os.getenv("AZURE_OPENAI_LLM"),
    deployment_name=os.getenv("AZURE_OPENAI_LLM"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
)

openai_embedding = AzureOpenAIEmbedding(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version= os.getenv("AZURE_OPENAI_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDER"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    model=os.getenv("AZURE_OPENAI_EMBEDDER"),
)

In [16]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core.schema import TextNode
from llama_index.embeddings.adapter import LinearAdapterEmbeddingModel
from llama_index.finetuning import (
    generate_qa_embedding_pairs, 
    EmbeddingAdapterFinetuneEngine,
    EmbeddingQAFinetuneDataset
)
from typing import List, Optional, Any
from tqdm.notebook import tqdm
import random
import pandas as pd

def train_val_test_split(
        document_store: MilvusDocumentStore, 
        split_ratio: List = [0.6, 0.2, 0.2], 
        seed: int = 42
    ) -> tuple[List[str], List[str], List[str]]:
    """
    Splits a collection of documents into training and validation sets based on a given ratio.

    Args:
        collection (list): A collection of documents to be split.
        split_ratio (float): The ratio of documents to be included in the training set (default: 0.8).

    Returns:
        tuple: A tuple containing the training and validation sets of documents.
    """
    random.seed(seed)
    collection = document_store.col
    sources = [doc["source"] for doc in collection.query(expr="id != ''", output_fields=["source"])]
    sources = list(set(sources))
    random.shuffle(sources)
    train_split_index = int(len(sources) * split_ratio[0])
    val_split_index = int(len(sources) * (split_ratio[0] + split_ratio[1]))
    train_set = sources[:train_split_index]
    val_set = sources[train_split_index:val_split_index]
    test_set = sources[val_split_index:]
    train_documents = collection.query(expr=f"source in {train_set}", output_fields=["text"])
    val_documents = collection.query(expr=f"source in {val_set}", output_fields=["text"])
    test_documents = collection.query(expr=f"source in {test_set}", output_fields=["text"])
    train_chunks = [(document["text"], document["id"]) for document in train_documents]
    val_chunks = [(document["text"], document['id']) for document in val_documents]
    test_chunks = [(document["text"], document['id']) for document in test_documents]
    return train_chunks, val_chunks, test_chunks

def get_query_chunk_dataset(
        chunks: List[str], 
        llm: Optional[Any] = None,
        json_path: Optional[str] = None,
        prompt: Optional[str] = None,
        verbose: bool = False
    ) -> EmbeddingQAFinetuneDataset:
    """
    Generates a dataset of question-answer pairs from a list of text chunks.

    Args:
        chunks (list): A list of text chunks to be converted into question-answer pairs.
        llm (AzureOpenAI): The language model to be used for generating the question-answer pairs (default: None).
        json_path (str): The file path to save the dataset as a JSON file (default: None).
        verbose (bool): If True, print debugging messages (default: False).
    
    Returns:
        EmbeddingQAFinetuneDataset: A dataset of question-answer pairs.
    """
    if llm is None:
        llm = llm
    chunk_nodes = [TextNode(text=chunk[0]) for chunk in chunks]
    if prompt is None:
        prompt = """\
        Context information is below.

        ---------------------
        {context_str}
        ---------------------

        Given the context information and no prior knowledge.
        generate only questions based on the below query.

        You are a Teacher/ Professor. Your task is to setup \
        {num_questions_per_chunk} questions for an upcoming \
        quiz/examination. The questions should be diverse in nature \
        across the document. Restrict the questions to the \
        context information provided. Give only the questions, \
        and no extra commentary, formatting, or chattiness.
        """
    dataset = generate_qa_embedding_pairs(
        chunk_nodes, 
        llm=llm, 
        verbose=verbose,
        output_path=json_path,
        qa_generate_prompt_tmpl=prompt
    )
    if json_path:
        dataset.save_json(json_path)
    return dataset

def train_linear_adapter(
        dataset: EmbeddingQAFinetuneDataset, 
        embedder_model: AzureOpenAIEmbedding, 
        epochs: int = 3, 
        model_output_path: str = "./models/linear_adapter",
        verbose: bool = True
    ) -> LinearAdapterEmbeddingModel:
    """
    Finetunes a linear adapter on a given dataset of question-answer pairs.

    Args:
        dataset (EmbeddingQAFinetuneDataset): The dataset of question-answer pairs to be used for finetuning.
        model_name (str): The name of the model to be used for finetuning (default: "microsoft/deberta-base").
        epochs (int): The number of epochs to run during finetuning (default: 3).
        batch_size (int): The batch size to use during finetuning (default: 8).

    Returns:
        None
    """
    if not os.path.exists(model_output_path):
        os.makedirs(model_output_path)
    finetune_engine = EmbeddingAdapterFinetuneEngine(
        dataset=dataset,
        embed_model=embedder_model,
        epochs=epochs,
        model_output_path=model_output_path,
        verbose=verbose
    )
    finetune_engine.finetune()
    return finetune_engine.get_finetuned_model()

def evaluate(dataset, embed_model, llm, top_k=5, verbose=False):
    """
    Evaluate models.
    """
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs
    
    # Replace ServiceContext with Settings
    Settings.llm = llm
    Settings.embed_model = embed_model
    
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, settings=Settings, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]

        rank = None
        for idx, id in enumerate(retrieved_ids):
            if id == expected_id:
                rank = idx + 1
                break

        is_hit = rank is not None  # assume 1 relevant doc
        mrr = 0 if rank is None else 1 / rank

        eval_result = {
            "is_hit": is_hit,
            "mrr": mrr,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    
    return eval_results

def display_results(names, results_arr):
    """
    Display results from evaluate.
    """
    hit_rates = []
    mrrs = []
    for name, results in zip(names, results_arr):
        results_df = pd.DataFrame(results)
        hit_rate = results_df["is_hit"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame({"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs})
    display(final_df)

In [6]:
train_set, val_set, test_set = train_val_test_split(document_store, split_ratio=[0.5, 0.5, 0])

In [8]:
print(train_set[0][0])

Blu-ray Disc allows video with a bit depth of 8-bits per color YCbCr with 4:2:0 chroma subsampling.[185][186] The choice of formats affects the producer's licensing/royalty costs as well as the title's maximum run time, due to differences in compression efficiency. 


In [19]:
train_set, val_set, test_set = train_val_test_split(document_store, split_ratio=[0.1, 0.1, 0.8])
train_dataset = get_query_chunk_dataset(train_set, llm_llama_index, './data/train.json', verbose=True)
val_dataset = get_query_chunk_dataset(val_set, llm_llama_index, './data/val.json')
trained_model = train_linear_adapter(train_dataset, openai_embedding, verbose=False)

655it [00:01,  1.59s/it]


Final dataset saved.


563it [00:00, ?it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/131 [00:00<?, ?it/s]

Iteration:   0%|          | 0/131 [00:00<?, ?it/s]

Iteration:   0%|          | 0/131 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/llama_index/embeddings/adapter/utils.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


In [20]:
eval_openai_results = evaluate(val_dataset, openai_embedding, llm_llama_index)
eval_trained_results = evaluate(val_dataset, trained_model, llm_llama_index)

Generating embeddings:   0%|          | 0/563 [00:00<?, ?it/s]

  0%|          | 0/1126 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/563 [00:00<?, ?it/s]

  0%|          | 0/1126 [00:00<?, ?it/s]

In [21]:
display_results(["openai", "trained_model"], [eval_openai_results, eval_trained_results]) 

,retrievers,hit_rate,mrr
0,openai,0.801066,0.719035
1,trained_model,0.814387,0.725429


## Testing class implementation with Dataset Generation

In [46]:
import sys
import os

# Get the current file's directory (e.g., the 'notebooks' directory)
current_dir = os.path.dirname(os.path.abspath(''))

# Navigate one level up
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the directory to sys.path
sys.path.append(parent_dir)

In [47]:
from src.evaluation.dataset_generation.dataset_generation import DatasetGenerator, myDataset

# Initialize the dataset generator
generator = DatasetGenerator(document_store=document_store, model=llm)
train_set, val_set, test_set, train_sources, val_sources, test_sources = generator.train_val_test_split(split_ratio=[0.1, 0.1, 0.8])

In [24]:
train_dataset = generator.generate_dataset(
    sources=train_sources,
    number_of_questions=5,
    chunks=train_set,
    generate_answers=True,
    get_multi_context=True,
    json_path='./train_multi.json',
    chunk_size_threshold=200,
    num_questions_per_context=1
)

Answering Queries: 100%|██████████| 5/5 [00:11<00:00,  2.37s/it]


In [25]:
expected_answers = train_dataset.expected_answers
queries = train_dataset.queries
corpus = train_dataset.corpus
relevant_docs = train_dataset.relevant_docs

for query_id, query in queries.copy().items():
    if query == "no answer":
        queries.pop(query_id)
        expected_answers.pop(query_id)
        relevant_docs.pop(query_id)

cleaned_train_dataset = myDataset(queries=queries, corpus=corpus, relevant_docs=relevant_docs, expected_answers=expected_answers)
cleaned_train_dataset.save_json("data/cleaned_train_multi.json")

In [26]:
val_dataset = generator.generate_dataset(
    sources=val_sources,
    number_of_questions=5,
    chunks=val_set,
    generate_answers=True,
    get_multi_context=True,
    json_path='./val_multi.json',
    chunk_size_threshold=200,
    num_questions_per_context=1
)

Answering Queries: 100%|██████████| 7/7 [00:48<00:00,  6.91s/it]


In [27]:
expected_answers = val_dataset.expected_answers
queries = val_dataset.queries
corpus = val_dataset.corpus
relevant_docs = val_dataset.relevant_docs

for query_id, query in queries.copy().items():
    if query == "no answer":
        queries.pop(query_id)
        expected_answers.pop(query_id)
        relevant_docs.pop(query_id)

cleaned_val_dataset = myDataset(queries=queries, corpus=corpus, relevant_docs=relevant_docs, expected_answers=expected_answers)
cleaned_val_dataset.save_json("data/cleaned_val_multi.json")

In [48]:
from src.components.linear_adapter import LinearAdapterTrainer
from src.evaluation.dataset_generation.dataset_generation import myDataset

train_dataset = myDataset.from_json('data/cleaned_train_multi.json')
trainer = LinearAdapterTrainer(embedding_model_name="azure", llm_model_name="azure")
trainer.train_linear_adapter(dataset=train_dataset, epochs=5, model_output_path="./models/linear_adapter_multi")

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.12/site-packages/llama_index/embeddings/adapter/utils.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


AdapterEmbeddingModel(model_name='Adapter for text-embedding-3-large', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x1238777a0>, num_workers=None)

In [49]:
from src.components.linear_adapter import LinearAdapter

adapter = LinearAdapter(model_name='azure', adapter_path='./models/linear_adapter_multi')
adapter.warm_up()
embeddings_linear_adapter = adapter.run("What is the capital of France?")
embeddings_openai = openai_embedding.get_query_embedding("What is the capital of France?")

In [50]:
print(embeddings_linear_adapter['embedding'])

[-0.05556660145521164, 0.04077581316232681, -0.01215212419629097, 0.04271968826651573, -0.05642217397689819, -0.027362056076526642, 0.022980140522122383, -0.01159043051302433, -0.02687087468802929, -0.00873649213463068, -0.002645015949383378, 0.00040371291106566787, -0.026368355378508568, -0.018258031457662582, 0.01223254855722189, 0.0006673458847217262, -0.00523545453324914, 0.016669418662786484, -8.85321423993446e-05, 0.00761406309902668, 0.02109498158097267, -0.008469895459711552, -0.019506093114614487, -0.038982391357421875, 0.00047443233779631555, 0.008836833760142326, 0.03859668970108032, -0.00918544176965952, 0.03550910949707031, 0.02871611714363098, 0.0736028403043747, 0.004224516451358795, 0.01729983650147915, -0.017494991421699524, -0.044418323785066605, -0.0043688989244401455, 0.015127199701964855, 0.015409138984978199, 0.025346146896481514, 0.0066240946762263775, 0.009717057459056377, -0.01219632476568222, 0.007185031659901142, 0.021228952333331108, 0.03218434378504753, -0.

In [51]:
print(embeddings_openai)

[-0.05562730133533478, 0.04083847999572754, -0.012112065218389034, 0.04262298345565796, -0.05648878589272499, -0.027403419837355614, 0.02297292649745941, -0.011609532870352268, -0.026952166110277176, -0.008630231022834778, -0.002653680741786957, 0.00045221540494821966, -0.026439378038048744, -0.01828604005277157, 0.012327436357736588, 0.0007666188757866621, -0.005317617207765579, 0.016563070937991142, -2.5779643692658283e-05, 0.007691828068345785, 0.021044841036200523, -0.008450754918158054, -0.0194551981985569, -0.03905397653579712, 0.0004070259165018797, 0.008937904611229897, 0.03852067515254021, -0.009107124991714954, 0.035402920097112656, 0.028716158121824265, 0.0735543891787529, 0.004174098838120699, 0.01739378832280636, -0.017424555495381355, -0.04438697546720505, -0.004461260512471199, 0.015055472031235695, 0.015322121791541576, 0.02522919699549675, 0.006538053974509239, 0.00981990061700344, -0.012101809494197369, 0.007122633047401905, 0.021249957382678986, 0.032121073454618454,